In [2]:
#!/usr/bin/env python3
import os
import numpy as np
from pathlib import Path
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# ---------------------------------------------------------
# Optional OpenSlide import
# ---------------------------------------------------------
try:
    import openslide
    HAS_OPENSLIDE = True
except ImportError:
    HAS_OPENSLIDE = False

# ---------------------------------------------------------
# Background filter
# ---------------------------------------------------------
def is_not_background(tile: Image.Image, bg_thresh=220, max_bg_pct=0.50) -> bool:
    arr = np.array(tile)
    if arr.ndim != 3:
        return False
    gray = arr.mean(axis=2)
    pct_background = np.mean(gray > bg_thresh)
    return pct_background <= max_bg_pct


# ---------------------------------------------------------
# DeepZoom coordinate generator
# ---------------------------------------------------------
def dz_coords(level_w: int, level_h: int, s: int, e: int):
    """
    Generate DeepZoom tile windows.
    Final tile size = s + 2e.
    """
    stride = s
    tile_full = s + 2*e
    cols = max(1, (level_w + stride - 1) // stride)
    rows = max(1, (level_h + stride - 1) // stride)

    for row in range(rows):
        for col in range(cols):
            x_in = col * stride
            y_in = row * stride

            x_in = min(x_in, level_w - s)
            y_in = min(y_in, level_h - s)

            x0 = max(0, x_in - e)
            y0 = max(0, y_in - e)
            x1 = min(level_w, x_in + s + e)
            y1 = min(level_h, y_in + s + e)

            yield col, row, x0, y0, x1, y1


# ---------------------------------------------------------
# Worker function
# ---------------------------------------------------------
def tile_worker(args):
    (slide_path, out_dir, level, s, e, bg_thresh, max_bg_pct) = args

    slide = openslide.OpenSlide(slide_path)
    level_w, level_h = slide.level_dimensions[level]
    ds = slide.level_downsamples[level]

    count = 0

    for col, row, x0_l, y0_l, x1_l, y1_l in dz_coords(level_w, level_h, s, e):
        # Convert from level coords → level 0 coords
        x0 = int(round(x0_l * ds))
        y0 = int(round(y0_l * ds))
        w = int(round(x1_l - x0_l))
        h = int(round(y1_l - y0_l))

        tile = slide.read_region((x0, y0), level, (w, h)).convert("RGB")

        if not is_not_background(tile, bg_thresh, max_bg_pct):
            continue

        tile.save(out_dir / f"{col}_{row}.png")
        count += 1

    slide.close()
    return count


# ---------------------------------------------------------
# Process one slide
# ---------------------------------------------------------
def process_slide(slide_path: str, output_root: Path,
                  level: int, s: int, e: int,
                  bg_thresh: int, max_bg_pct: float):

    slide_path = Path(slide_path)

    out_dir = output_root / slide_path.stem
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f" → Tiling: {slide_path.name}")

    args = (
        str(slide_path),
        out_dir,
        level,
        s,
        e,
        bg_thresh,
        max_bg_pct,
    )

    count = tile_worker(args)
    print(f"    Saved {count} tiles.\n")


# ---------------------------------------------------------
# Run all slides from CSV
# ---------------------------------------------------------
def run_csv(csv_path: Path, data_dir: Path, output_dir: Path,
            level: int, s: int, e: int, bg_thresh: int, max_bg_pct: float):
    import pandas as pd

    df = pd.read_csv(csv_path)

    # Resolve image IDs
    if "image_id" in df.columns:
        ids = df["image_id"].astype(str).tolist()
    else:
        ids = df.iloc[:, 0].astype(str).tolist()

    print(f"Loaded {len(ids)} IDs.")

    extensions = [".tiff", ".tif", ".svs"]

    found = []
    missing = []

    for img_id in ids:
        match = None
        for ext in extensions:
            p = data_dir / f"{img_id}{ext}"
            if p.exists():
                match = p
                break
        if match:
            found.append(match)
        else:
            missing.append(img_id)

    print(f"Found   : {len(found)} slides.")
    print(f"Missing : {len(missing)}\n")

    for i, slide_path in enumerate(found, 1):
        print(f"[{i}/{len(found)}] {slide_path.name}")

        process_slide(
            slide_path,
            output_dir,
            level,
            s,
            e,
            bg_thresh,
            max_bg_pct
        )

    print("✓ All slides processed.")


# ---------------------------------------------------------
# MAIN ENTRY — HARD-CODED PATHS
# ---------------------------------------------------------
def main():

    # === HARD-CODED PATHS (SCC) ===
    csv_path   = Path("Official_GTP_PANDAS/PANDAS/data/subsets/train_patch_10.csv")
    data_dir   = Path("PANDA_DATA_MANNY/DATA/train_images")
    output_dir = Path("PANDA_DATA_MANNY/PATCHES_SUBSET_10")

    # === TILE PARAMS ===
    tile_size    = 512
    tile_overlap = 0
    level        = 1
    bg_thresh    = 220
    max_bg_pct   = 0.50

    print("=== WSI PATCH EXTRACTION STARTED ===")
    print(f"CSV        : {csv_path}")
    print(f"Data dir   : {data_dir}")
    print(f"Output dir : {output_dir}\n")

    run_csv(
        csv_path=csv_path,
        data_dir=data_dir,
        output_dir=output_dir,
        level=level,
        s=tile_size,
        e=tile_overlap,
        bg_thresh=bg_thresh,
        max_bg_pct=max_bg_pct,
    )

    print("=== DONE ===")


# ---------------------------------------------------------
# Run
# ---------------------------------------------------------
if __name__ == "__main__":
    main()


=== WSI PATCH EXTRACTION STARTED ===
CSV        : Official_GTP_PANDAS/PANDAS/data/subsets/train_patch_10.csv
Data dir   : PANDA_DATA_MANNY/DATA/train_images
Output dir : PANDA_DATA_MANNY/PATCHES_SUBSET_10

Loaded 842 IDs.
Found   : 842 slides.
Missing : 0

[1/842] 99cc5f5b9f1a86c14dc1970defb07799.tiff
 → Tiling: 99cc5f5b9f1a86c14dc1970defb07799.tiff
    Saved 5 tiles.

[2/842] 72b46ce40040292c972e5c8fc84f857d.tiff
 → Tiling: 72b46ce40040292c972e5c8fc84f857d.tiff
    Saved 3 tiles.

[3/842] 0356fabdec47163b6ad35a2a7b06647c.tiff
 → Tiling: 0356fabdec47163b6ad35a2a7b06647c.tiff
    Saved 5 tiles.

[4/842] b97714e3e07e57a9bfc005fc830a1723.tiff
 → Tiling: b97714e3e07e57a9bfc005fc830a1723.tiff
    Saved 7 tiles.

[5/842] 988c9deb4dc9996464fbccb6976dad1c.tiff
 → Tiling: 988c9deb4dc9996464fbccb6976dad1c.tiff
    Saved 3 tiles.

[6/842] 20fe45e5ea3022602775d0aa86e8ef94.tiff
 → Tiling: 20fe45e5ea3022602775d0aa86e8ef94.tiff
    Saved 1 tiles.

[7/842] 31a2c62376a98ed5062bbe72a92b3541.tiff
 → Tili

    Saved 2 tiles.

[70/842] 98bfad4a1ec921edc2f61c9c9dd5690a.tiff
 → Tiling: 98bfad4a1ec921edc2f61c9c9dd5690a.tiff
    Saved 5 tiles.

[71/842] b267fa5f2c3b44774f06581ac05996b7.tiff
 → Tiling: b267fa5f2c3b44774f06581ac05996b7.tiff
    Saved 3 tiles.

[72/842] bda25c48ac8086cccbf2397e13166046.tiff
 → Tiling: bda25c48ac8086cccbf2397e13166046.tiff
    Saved 6 tiles.

[73/842] 135e69f222d3c272fd97a5914341065f.tiff
 → Tiling: 135e69f222d3c272fd97a5914341065f.tiff
    Saved 0 tiles.

[74/842] 99061fca83e817915e3e20536bf9fe39.tiff
 → Tiling: 99061fca83e817915e3e20536bf9fe39.tiff
    Saved 5 tiles.

[75/842] 30214f9a2c610f550a40501fd11d74b1.tiff
 → Tiling: 30214f9a2c610f550a40501fd11d74b1.tiff
    Saved 4 tiles.

[76/842] 5f40e0b339592b1b70f17134a06abf98.tiff
 → Tiling: 5f40e0b339592b1b70f17134a06abf98.tiff
    Saved 5 tiles.

[77/842] 9962fd78160191c674d0032acc13c4a7.tiff
 → Tiling: 9962fd78160191c674d0032acc13c4a7.tiff
    Saved 1 tiles.

[78/842] 6ad5640ca76614728d3b699a703ef3c8.tiff
 → Ti

    Saved 9 tiles.

[141/842] f35ee5225fcd673e2e5ed50ab85cb43d.tiff
 → Tiling: f35ee5225fcd673e2e5ed50ab85cb43d.tiff
    Saved 1 tiles.

[142/842] adae03ea908e72c0e179f10f8df094ef.tiff
 → Tiling: adae03ea908e72c0e179f10f8df094ef.tiff
    Saved 1 tiles.

[143/842] aed7330d040f019f279d673133cbfe53.tiff
 → Tiling: aed7330d040f019f279d673133cbfe53.tiff
    Saved 13 tiles.

[144/842] 2530082d8f4b3eee15503339398d7fb3.tiff
 → Tiling: 2530082d8f4b3eee15503339398d7fb3.tiff
    Saved 4 tiles.

[145/842] 932af254f55ed4607b6a31c4fc69726f.tiff
 → Tiling: 932af254f55ed4607b6a31c4fc69726f.tiff
    Saved 0 tiles.

[146/842] ef04092847409d2ea98b392331985dcb.tiff
 → Tiling: ef04092847409d2ea98b392331985dcb.tiff
    Saved 6 tiles.

[147/842] 320a5a8ed48d1d03103a79659913c987.tiff
 → Tiling: 320a5a8ed48d1d03103a79659913c987.tiff
    Saved 5 tiles.

[148/842] c5a32c937e25c87604d960166767411e.tiff
 → Tiling: c5a32c937e25c87604d960166767411e.tiff
    Saved 0 tiles.

[149/842] 0a81500dc48b2bb7ee19849502360270.

    Saved 2 tiles.

[211/842] 5479baf96910f3cf404a133651a38b3e.tiff
 → Tiling: 5479baf96910f3cf404a133651a38b3e.tiff
    Saved 6 tiles.

[212/842] 1fd3d60b4833d266003fce5f17205baa.tiff
 → Tiling: 1fd3d60b4833d266003fce5f17205baa.tiff
    Saved 4 tiles.

[213/842] 97bed3de5c372f3dc9ba818cd5e97369.tiff
 → Tiling: 97bed3de5c372f3dc9ba818cd5e97369.tiff
    Saved 3 tiles.

[214/842] e4c2c0efe450797793cc090b2fd59f84.tiff
 → Tiling: e4c2c0efe450797793cc090b2fd59f84.tiff
    Saved 3 tiles.

[215/842] 842d323ec987a756ba0c4252be109822.tiff
 → Tiling: 842d323ec987a756ba0c4252be109822.tiff
    Saved 5 tiles.

[216/842] 67bc9a23a87b07457464561b22515733.tiff
 → Tiling: 67bc9a23a87b07457464561b22515733.tiff
    Saved 5 tiles.

[217/842] 79212be3df3e81973f9e0365920f2282.tiff
 → Tiling: 79212be3df3e81973f9e0365920f2282.tiff
    Saved 6 tiles.

[218/842] 2ed1c3b9541f619a827a287a55c806e8.tiff
 → Tiling: 2ed1c3b9541f619a827a287a55c806e8.tiff
    Saved 5 tiles.

[219/842] 9aa47821e79af6bbacc6ee49f915156c.t

    Saved 3 tiles.

[282/842] 83f3b246bdbd51ed830877c3991bf7ca.tiff
 → Tiling: 83f3b246bdbd51ed830877c3991bf7ca.tiff
    Saved 19 tiles.

[283/842] b202b3ae4299e8b1264e3298e9022e88.tiff
 → Tiling: b202b3ae4299e8b1264e3298e9022e88.tiff
    Saved 7 tiles.

[284/842] e2578d331ae6cb3bc649027498c8ff8d.tiff
 → Tiling: e2578d331ae6cb3bc649027498c8ff8d.tiff
    Saved 5 tiles.

[285/842] 75272c0e3fc6faeb17a353b704051e11.tiff
 → Tiling: 75272c0e3fc6faeb17a353b704051e11.tiff
    Saved 0 tiles.

[286/842] 9fa1b4e617cbc88646794a6204e2d897.tiff
 → Tiling: 9fa1b4e617cbc88646794a6204e2d897.tiff
    Saved 3 tiles.

[287/842] 834654070eb44fb589d68557f0f48587.tiff
 → Tiling: 834654070eb44fb589d68557f0f48587.tiff
    Saved 0 tiles.

[288/842] 7fa9e5162e347814fdf3596f3ebcedd8.tiff
 → Tiling: 7fa9e5162e347814fdf3596f3ebcedd8.tiff
    Saved 7 tiles.

[289/842] be56b8fd2f70cb10e72deae6baedcd51.tiff
 → Tiling: be56b8fd2f70cb10e72deae6baedcd51.tiff
    Saved 8 tiles.

[290/842] 3600b9f393854a0c456904beaff22ae9.

...

    Saved 6 tiles.

[494/842] 0403dcc49b1420545299f692f7d8e270.tiff
 → Tiling: 0403dcc49b1420545299f692f7d8e270.tiff
    Saved 5 tiles.

[495/842] bd465b5e566f6336037c548644e11e16.tiff
 → Tiling: bd465b5e566f6336037c548644e11e16.tiff
    Saved 1 tiles.

[496/842] 4b6ed79396add87dafb5a4d6846c6972.tiff
 → Tiling: 4b6ed79396add87dafb5a4d6846c6972.tiff
    Saved 2 tiles.

[497/842] 2a4cff606b103cb1698560ba1dee0eb6.tiff
 → Tiling: 2a4cff606b103cb1698560ba1dee0eb6.tiff
    Saved 2 tiles.

[498/842] 63af8bc50ca5f1f8b8aca0a46a7fc428.tiff
 → Tiling: 63af8bc50ca5f1f8b8aca0a46a7fc428.tiff
    Saved 6 tiles.

[499/842] b49d54b4f7e73808440c36fa240614c0.tiff
 → Tiling: b49d54b4f7e73808440c36fa240614c0.tiff
    Saved 6 tiles.

[500/842] 6071aa8ba6c71f822201674f662a2058.tiff
 → Tiling: 6071aa8ba6c71f822201674f662a2058.tiff
    Saved 1 tiles.

[501/842] 448886cc21e672bbf711badd2d5b01da.tiff
 → Tiling: 448886cc21e672bbf711badd2d5b01da.tiff
    Saved 1 tiles.

[502/842] 2fd1c7dc4a0f3a546a59717d8e9d28c3.t

    Saved 2 tiles.

[564/842] c970573e9cb96664f92635e105212888.tiff
 → Tiling: c970573e9cb96664f92635e105212888.tiff
    Saved 1 tiles.

[565/842] 184b98556b06d9962dee7e737b23902b.tiff
 → Tiling: 184b98556b06d9962dee7e737b23902b.tiff
    Saved 2 tiles.

[566/842] 47f34de873955e8e03e954e3882d747f.tiff
 → Tiling: 47f34de873955e8e03e954e3882d747f.tiff
    Saved 5 tiles.

[567/842] 6dcece3682e1f8d8772be8dd3649b1ab.tiff
 → Tiling: 6dcece3682e1f8d8772be8dd3649b1ab.tiff
    Saved 0 tiles.

[568/842] 7acd16338dbb2d1fa02cd6f3b80436a3.tiff
 → Tiling: 7acd16338dbb2d1fa02cd6f3b80436a3.tiff
    Saved 0 tiles.

[569/842] 4a2ca53f240932e46eaf8959cb3f490a.tiff
 → Tiling: 4a2ca53f240932e46eaf8959cb3f490a.tiff
    Saved 3 tiles.

[570/842] 36eeee47a868b8ac66b6ece9e6372984.tiff
 → Tiling: 36eeee47a868b8ac66b6ece9e6372984.tiff
    Saved 1 tiles.

[571/842] 028098c36eb49a8c6aa6e76e365dd055.tiff
 → Tiling: 028098c36eb49a8c6aa6e76e365dd055.tiff
    Saved 2 tiles.

[572/842] fc860f59830b4295eaa563554b16458a.t

    Saved 3 tiles.

[634/842] 6ccd71f80656fd3329eb9547d783fb8f.tiff
 → Tiling: 6ccd71f80656fd3329eb9547d783fb8f.tiff
    Saved 5 tiles.

[635/842] d887469d2d1a490b480e32c145c68469.tiff
 → Tiling: d887469d2d1a490b480e32c145c68469.tiff
    Saved 1 tiles.

[636/842] 68f4d04f60368beedcf85796393f45c9.tiff
 → Tiling: 68f4d04f60368beedcf85796393f45c9.tiff
    Saved 4 tiles.

[637/842] 055dc74f97a387fd1f12beb64e1cca76.tiff
 → Tiling: 055dc74f97a387fd1f12beb64e1cca76.tiff
    Saved 1 tiles.

[638/842] 57d82e5381a64e28e1daae58c9806ab5.tiff
 → Tiling: 57d82e5381a64e28e1daae58c9806ab5.tiff
    Saved 2 tiles.

[639/842] 79b3f2787261e5c96b3c9bf9a33c7537.tiff
 → Tiling: 79b3f2787261e5c96b3c9bf9a33c7537.tiff
    Saved 5 tiles.

[640/842] 880c1f78ee7e01cee8776b5ec789bd80.tiff
 → Tiling: 880c1f78ee7e01cee8776b5ec789bd80.tiff
    Saved 3 tiles.

[641/842] f066af8d3d3f1eb3471b817d757f370d.tiff
 → Tiling: f066af8d3d3f1eb3471b817d757f370d.tiff
    Saved 6 tiles.

[642/842] 8e18a003593e835063a89a8709df3979.t

    Saved 0 tiles.

[705/842] d34c7fe759808771f0b2dc65ad0df03c.tiff
 → Tiling: d34c7fe759808771f0b2dc65ad0df03c.tiff
    Saved 0 tiles.

[706/842] b399c54dd9f6ee4add1b4a21dbdbf6f6.tiff
 → Tiling: b399c54dd9f6ee4add1b4a21dbdbf6f6.tiff
    Saved 6 tiles.

[707/842] 45f109ea3026f40eb2d44209791eb3ca.tiff
 → Tiling: 45f109ea3026f40eb2d44209791eb3ca.tiff
    Saved 6 tiles.

[708/842] 15b76986fa6a5826ed39c184aea31e4f.tiff
 → Tiling: 15b76986fa6a5826ed39c184aea31e4f.tiff
    Saved 1 tiles.

[709/842] be4ed7eb6be88b0c859a7dbdea0c90fc.tiff
 → Tiling: be4ed7eb6be88b0c859a7dbdea0c90fc.tiff
    Saved 5 tiles.

[710/842] 25ab0410a69c9d163d0371bd82b2291a.tiff
 → Tiling: 25ab0410a69c9d163d0371bd82b2291a.tiff
    Saved 1 tiles.

[711/842] ab53934e13fe844e43c6c903dabf0128.tiff
 → Tiling: ab53934e13fe844e43c6c903dabf0128.tiff
    Saved 2 tiles.

[712/842] bb2087bf01f0ef288c6e2fdb83b6b283.tiff
 → Tiling: bb2087bf01f0ef288c6e2fdb83b6b283.tiff
    Saved 6 tiles.

[713/842] 92d5a8e66feaffb190bdf88ea20cff2d.t

    Saved 1 tiles.

[775/842] a7542dcc3123f9cce837cd93f13c0a27.tiff
 → Tiling: a7542dcc3123f9cce837cd93f13c0a27.tiff
    Saved 2 tiles.

[776/842] eaeb7f15cb0618d6091ef52e685ec284.tiff
 → Tiling: eaeb7f15cb0618d6091ef52e685ec284.tiff
    Saved 8 tiles.

[777/842] c14beccd6b55d7b57cbc2304987b324b.tiff
 → Tiling: c14beccd6b55d7b57cbc2304987b324b.tiff
    Saved 6 tiles.

[778/842] 5ffcf2372542336fe4ee3585bd07b6f2.tiff
 → Tiling: 5ffcf2372542336fe4ee3585bd07b6f2.tiff
    Saved 7 tiles.

[779/842] 1a33d8ea1893170a972d67e493d59396.tiff
 → Tiling: 1a33d8ea1893170a972d67e493d59396.tiff
    Saved 3 tiles.

[780/842] 4d048dd585259614c27e11431ec2c860.tiff
 → Tiling: 4d048dd585259614c27e11431ec2c860.tiff
    Saved 10 tiles.

[781/842] b6dbb1d1ef20d9d91326e8d9c9493c31.tiff
 → Tiling: b6dbb1d1ef20d9d91326e8d9c9493c31.tiff
    Saved 9 tiles.

[782/842] 622fc40ffe3f5307f850d3e6e3e738a4.tiff
 → Tiling: 622fc40ffe3f5307f850d3e6e3e738a4.tiff
    Saved 6 tiles.

[783/842] 283d925bbde649adc0a7c703049ee7d5.

In [3]:
### Create CSV for SimCLR

In [4]:
import os
import pandas as pd

base = "PANDA_DATA_MANNY"
subsets = [f"PATCHES_SUBSET_{i:02d}" for i in range(1, 11)]

paths = []

for ss in subsets:
    root = os.path.join(base, ss)
    for slide in os.listdir(root):
        slide_path = os.path.join(root, slide)
        if not os.path.isdir(slide_path):
            continue
        for f in os.listdir(slide_path):
            if f.endswith(".png"):
                paths.append(os.path.join(slide_path, f))

df = pd.DataFrame({"path": paths})
df.to_csv("all_patches.csv", index=False)

print("Wrote", len(df), "patch paths.")


Wrote 32302 patch paths.
